In [1]:
# import liberies
import pandas as pd
import numpy as np

In [2]:
path = "C:/Users/DEEPAK KUMAR/Desktop/Python Project/bnp-paribas-cardif/train.csv"
train = pd.read_csv(path)

In [3]:
train.head(5)

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,...,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN


In [4]:
!pip install -q git+https://github.com/mljar/mljar-supervised.git@dev

  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git clone -q https://github.com/mljar/mljar-supervised.git 'C:\Users\DEEPAK KUMAR\AppData\Local\Temp\pip-req-build-_9tphfqf'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [5]:
import logging

from supervised.base_automl import BaseAutoML

from supervised.utils.config import LOG_LEVEL


logging.basicConfig(
    format="%(asctime)s %(name)s %(levelname)s %(message)s", level=logging.ERROR
)
logger = logging.getLogger(__name__)
logger.setLevel(LOG_LEVEL)


class AutoML(BaseAutoML):

    """
    Automated Machine Learning for supervised tasks (binary classification, multiclass classification, regression).
    """

    def __init__(
        self,
        results_path=None,
        total_time_limit=60 * 60,
        mode="Explain",
        ml_task="auto",
        model_time_limit=None,
        algorithms="auto",
        train_ensemble=True,
        stack_models="auto",
        eval_metric="auto",
        validation_strategy="auto",
        explain_level="auto",
        golden_features="auto",
        features_selection="auto",
        start_random_models="auto",
        hill_climbing_steps="auto",
        top_models_to_improve="auto",
        boost_on_errors="auto",
        kmeans_features="auto",
        mix_encoding="auto",
        max_single_prediction_time=None,
        optuna_time_budget=None,
        optuna_init_params={},
        optuna_verbose=True,
        n_jobs=-1,
        verbose=1,
        random_state=1234,
    ):
        """
        Initialize `AutoML` object.
        Arguments:
            results_path (str): The path with results. If None, then the name of directory will be generated with the template: AutoML_{number},
                where the number can be from 1 to 1,000 - depends which direcory name will be available.
                If the `results_path` will point to directory with AutoML results (`params.json` must be present),
                then all models will be loaded.
            total_time_limit (int): The total time limit in seconds for AutoML training.
                It is not used when `model_time_limit` is not `None`.
            mode (str): Can be {`Explain`, `Perform`, `Compete`, `Optuna`}. This parameter defines the goal of AutoML and how intensive the AutoML search will be.
                - `Explain` : To to be used when the user wants to explain and understand the data.
                    - Uses 75%/25% train/test split.
                    - Uses the following models: `Baseline`, `Linear`, `Decision Tree`, `Random Forest`, `XGBoost`, `Neural Network`, and `Ensemble`.
                    - Has full explanations in reports: learning curves, importance plots, and SHAP plots.
                - `Perform` : To be used when the user wants to train a model that will be used in real-life use cases.
                    - Uses 5-fold CV (Cross-Validation).
                    - Uses the following models: `Linear`, `Random Forest`, `LightGBM`, `XGBoost`, `CatBoost`, `Neural Network`, and `Ensemble`.
                    - Has learning curves and importance plots in reports.
                - `Compete` : To be used for machine learning competitions (maximum performance).
                    - Uses 80/20 train/test split, or 5-fold CV, or 10-fold CV (Cross-Validation) - it depends on `total_time_limit`. If not set directly, AutoML will select validation automatically.
                    - Uses the following models: `Decision Tree`, `Random Forest`, `Extra Trees`, `LightGBM`,  `XGBoost`, `CatBoost`, `Neural Network`,
                        `Nearest Neighbors`, `Ensemble`, and `Stacking`.
                    - It has only learning curves in the reports.
                - `Optuna` : To be used for creating highly-tuned machine learning models.
                    - Uses 10-fold CV (Cross-Validation).
                    - It tunes with Optuna the following algorithms: `Random Forest`, `Extra Trees`, `LightGBM`, `XGBoost`, `CatBoost`, `Neural Network`.
                    - It applies `Ensemble` and `Stacking` for trained models.
                    - It has only learning curves in the reports.
            ml_task (str): Can be {"auto", "binary_classification", "multiclass_classification", "regression"}.
                - If left `auto` AutoML will try to guess the task based on target values.
                - If there will be only 2 values in the target, then task will be set to `"binary_classification"`.
                - If number of values in the target will be between 2 and 20 (included), then task will be set to `"multiclass_classification"`.
                - In all other casses, the task is set to `"regression"`.
            model_time_limit (int): The time limit for training a single model, in seconds.
                If `model_time_limit` is set, the `total_time_limit` is not respected.
                The single model can contain several learners. The time limit for subsequent learners is computed based on `model_time_limit`.
                For example, in the case of 10-fold cross-validation, one model will have 10 learners.
                The `model_time_limit` is the time for all 10 learners.
            algorithms (list of str): The list of algorithms that will be used in the training.
                The algorithms can be:
                - `Baseline`,
                - `Linear`,
                - `Decision Tree`,
                - `Random Forest`,
                - `Extra Trees`,
                - `LightGBM`,
                - `Xgboost`,
                - `CatBoost`,
                - `Neural Network`,
                - `Nearest Neighbors`,
            train_ensemble (boolean): Whether an ensemble gets created at the end of the training.
            stack_models (boolean): Whether a models stack gets created at the end of the training. Stack level is 1.
            eval_metric (str): The metric to be used in early stopping and to compare models.
                - for binary classification: `logloss`, `auc`, `f1`, `average_precision`, `accuracy` - default is logloss (if left "auto")
                - for mutliclass classification: `logloss`, `f1`, `accuracy` - default is `logloss` (if left "auto")
                - for regression: `rmse`, `mse`, `mae`, `r2`, `mape`, `spearman`, `pearson` - default is `rmse` (if left "auto")
            validation_strategy (dict): Dictionary with validation type. Right now train/test split and cross-validation are supported.
                Example:
                    Cross-validation exmaple:
                    {
                        "validation_type": "kfold",
                        "k_folds": 5,
                        "shuffle": True,
                        "stratify": True,
                        "random_seed": 123
                    }
                    Train/test example:
                    {
                        "validation_type": "split",
                        "train_ratio": 0.75,
                        "shuffle": True,
                        "stratify": True
                    }
            explain_level (int): The level of explanations included to each model:
                - if `explain_level` is `0` no explanations are produced.
                - if `explain_level` is `1` the following explanations are produced: importance plot (with permutation method), for decision trees produce tree plots, for linear models save coefficients.
                - if `explain_level` is `2` the following explanations are produced: the same as `1` plus SHAP explanations.
                If left `auto` AutoML will produce explanations based on the selected `mode`.
            golden_features (boolean or int): Whether to use golden features (and how many should be added)
                If left `auto` AutoML will use golden features based on the selected `mode`:
                - If `mode` is "Explain", `golden_features` = False.
                - If `mode` is "Perform", `golden_features` = True.
                - If `mode` is "Compete", `golden_features` = True.
                If `boolean` value is set then the number of Golden Features is set automatically.
                It is set to min(100, max(10, 0.1*number_of_input_features)).
                If `int` value is set, the number of Golden Features is set to this value.
            features_selection (boolean): Whether to do features_selection
                If left `auto` AutoML will do feature selection based on the selected `mode`:
                - If `mode` is "Explain", `features_selection` = False.
                - If `mode` is "Perform", `features_selection` = True.
                - If `mode` is "Compete", `features_selection` = True.
            start_random_models (int): Number of starting random models to try.
                If left `auto` AutoML will select it based on the selected `mode`:
                - If `mode` is "Explain", `start_random_models` = 1.
                - If `mode` is "Perform", `start_random_models` = 5.
                - If `mode` is "Compete", `start_random_models` = 10.
            hill_climbing_steps (int): Number of steps to perform during hill climbing.
                If left `auto` AutoML will select it based on the selected `mode`:
                - If `mode` is "Explain", `hill_climbing_steps` = 0.
                - If `mode` is "Perform", `hill_climbing_steps` = 2.
                - If `mode` is "Compete", `hill_climbing_steps` = 2.
            top_models_to_improve (int): Number of best models to improve in `hill_climbing` steps.
                If left `auto` AutoML will select it based on the selected `mode`:
                - If `mode` is "Explain", `top_models_to_improve` = 0.
                - If `mode` is "Perform", `top_models_to_improve` = 2.
                - If `mode` is "Compete", `top_models_to_improve` = 3.
            boost_on_errors (boolean): Whether a model with boost on errors from previous best model should be trained. By default available in the `Compete` mode.
            kmeans_features (boolean): Whether a model with k-means generated features should be trained. By default available in the `Compete` mode.
            mix_encoding (boolean): Whether a model with mixed encoding should be trained. Mixed encoding is the encoding that uses label encoding
                for categoricals with more than 25 categories, and one-hot binary encoding for other categoricals. It is only applied if there are
                categorical features with cardinality smaller than 25. By default it is available in the `Compete` mode.
            max_single_prediction_time (int or float): The limit for prediction time for single sample. Use it if you want to have a model with fast predictions.
                Ideal for creating ML pipelines used as REST API. Time is in seconds. By default (`max_single_prediction_time=None`) models are not optimized for fast predictions,
                except the mode `Perform`. For the mode `Perform` the default is `0.5` seconds.
            optuna_time_budget (int): The time in seconds which should be used by Optuna to tune each algorithm. It is time for tuning single algorithm.
                If you select two algorithms: Xgboost and CatBoost, and set optuna_time_budget=1000, then Xgboost will be tuned for 1000 seconds and CatBoost will be tuned for 1000 seconds.
                What is more, the tuning is made for each data type, for example for raw data and for data with inserted Golden Features.
                This parameter is only used when `mode="Optuna"`. If you set `mode="Optuna"` and forget to set this parameter, it will be set to 3600 seconds.
            optuna_init_params (dict): If you have already tuned parameters from Optuna you can reuse them by setting this parameter.
                This parameter is only used when `mode="Optuna"`. The dict should have structure and params as specified in the MLJAR AutoML .
            optuna_verbose (boolean): If true the Optuna tuning details are displayed. Set to `True` by default.
            n_jobs (int): Number of CPU cores to be used. By default is set to `-1` which means using  all processors.
            verbose (int): Controls the verbosity when fitting and predicting.
                Note:
                    Still not implemented, please left `1`
            random_state (int): Controls the randomness of the `AutoML`
        Examples:
            Binary Classification Example:
            >>> import pandas as pd
            >>> from sklearn.model_selection import train_test_split
            >>> from sklearn.metrics import roc_auc_score
            >>> from supervised import AutoML
            >>> df = pd.read_csv(
            ...        "https://raw.githubusercontent.com/pplonski/datasets-for-start/master/adult/data.csv",
            ...       skipinitialspace=True
            ...    )
            >>> X_train, X_test, y_train, y_test = train_test_split(
            ... df[df.columns[:-1]], df["income"], test_size=0.25
            ... )
            >>> automl = AutoML()
            >>> automl.fit(X_train, y_train)
            >>> y_pred_prob = automl.predict_proba(X_test)
            >>> print(f"AUROC: {roc_auc_score(y_test, y_pred_prob):.2f}%")
            Multi-Class Classification Example:
            >>> import pandas as pd
            >>> from sklearn.datasets import load_digits
            >>> from sklearn.metrics import accuracy_score
            >>> from sklearn.model_selection import train_test_split
            >>> from supervised import AutoML
            >>> digits = load_digits()
            >>> X_train, X_test, y_train, y_test = train_test_split(
            ...     digits.data, digits.target, stratify=digits.target, test_size=0.25,
            ...     random_state=123
            ... )
            >>> automl = AutoML(mode="Perform")
            >>> automl.fit(X_train, y_train)
            >>> y_pred = automl.predict(X_test)
            >>> print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}%")
            Regression Example:
            >>> import pandas as pd
            >>> from sklearn.datasets import load_boston
            >>> from sklearn.model_selection import train_test_split
            >>> from sklearn.metrics import mean_squared_error
            >>> from supervised import AutoML
            >>> housing = load_boston()
            >>> X_train, X_test, y_train, y_test = train_test_split(
            ...       pd.DataFrame(housing.data, columns=housing.feature_names),
            ...       housing.target,
            ...       test_size=0.25,
            ...       random_state=123,
            ... )
            >>> automl = AutoML(mode="Compete")
            >>> automl.fit(X_train, y_train)
            >>> print("Test R^2:", automl.score(X_test, y_test))
            Scikit-learn Pipeline Integration Example:
            >>> from imblearn.over_sampling import RandomOverSampler
            >>> from sklearn.pipeline import make_pipeline
            >>> from sklearn.datasets import make_classification
            >>> from sklearn.model_selection import train_test_split
            >>> from supervised import AutoML
            >>> X, y = make_classification()
            >>> X_train, X_test, y_train, y_test = train_test_split(X,y)
            >>> pipeline = make_pipeline(RandomOverSampler(), AutoML())
            >>> print(pipeline.fit(X_train, y_train).score(X_test, y_test))
        """
        super(AutoML, self).__init__()
        # Set user arguments
        self.mode = mode
        self.ml_task = ml_task
        self.results_path = results_path
        self.total_time_limit = total_time_limit
        self.model_time_limit = model_time_limit
        self.algorithms = algorithms
        self.train_ensemble = train_ensemble
        self.stack_models = stack_models
        self.eval_metric = eval_metric
        self.validation_strategy = validation_strategy
        self.verbose = verbose
        self.explain_level = explain_level
        self.golden_features = golden_features
        self.features_selection = features_selection
        self.start_random_models = start_random_models
        self.hill_climbing_steps = hill_climbing_steps
        self.top_models_to_improve = top_models_to_improve
        self.boost_on_errors = boost_on_errors
        self.kmeans_features = kmeans_features
        self.mix_encoding = mix_encoding
        self.max_single_prediction_time = max_single_prediction_time
        self.optuna_time_budget = optuna_time_budget
        self.optuna_init_params = optuna_init_params
        self.optuna_verbose = optuna_verbose
        self.n_jobs = n_jobs
        self.random_state = random_state

    def fit(self, X, y, sample_weight=None, cv=None):
        """Fit the AutoML model.
        Arguments:
            X (numpy.ndarray or pandas.DataFrame): Training data
            y (numpy.ndarray or pandas.Series): Training targets
            sample_weight (numpy.ndarray or pandas.Series): Training sample weights
            cv (iterable or list): List or iterable with (train, validation) splits representing array of indices.
            It is used only with custom validation (`validation_strategy={'validation_type': 'custom'}`).
        Returns:
            AutoML object: Returns `self`
        """
        return self._fit(X, y, sample_weight, cv)

    def predict(self, X):
        """
        Computes predictions from AutoML best model.
        Arguments:
            X (list or numpy.ndarray or pandas.DataFrame):
                Input values to make predictions on.
        Returns:
            numpy.ndarray:
            - One-dimensional array of class labels for classification.
            - One-dimensional array of predictions for regression.
        Raises:
            AutoMLException: Model has not yet been fitted.
        """
        return self._predict(X)

    def predict_proba(self, X):
        """
        Computes class probabilities from AutoML best model.
        This method can only be used for classification tasks.
        Arguments:
            X (list or numpy.ndarray or pandas.DataFrame):
                Input values to make predictions on.
        Returns:
            numpy.ndarray of shape (n_samples, n_classes):
                Matrix of containing class probabilities of the input samples
        Raises:
            AutoMLException: Model has not yet been fitted.
        """
        return self._predict_proba(X)

    def predict_all(self, X):
        """
        Computes both class probabilities and class labels for classification tasks.
        Computes predictions for regression tasks.
        Arguments:
            X (list or numpy.ndarray or pandas.DataFrame):
                Input values to make predictions on.
        Returns:
            pandas.Dataframe:
                Dataframe (n_samples, n_classes + 1) containing both class probabilities and class
                labels of the input samples for classification tasks.
                Dataframe with predictions for regression tasks.
        Raises:
            AutoMLException: Model has not yet been fitted.
        """
        return self._predict_all(X)

    def score(self, X, y=None, sample_weight=None):
        """Calculates a goodness of `fit` for an AutoML instance.
        Arguments:
            X (numpy.ndarray or pandas.DataFrame):
                Test values to make predictions on.
            y (numpy.ndarray or pandas.Series):
                True labels for X.
            sample_weight (numpy.ndarray or pandas.Series):
                Sample weights.
        Returns:
            float: Returns a goodness of fit measure (higher is better):
            - For classification tasks: returns the mean accuracy on the given test data and labels.
            - For regression tasks: returns the R^2 (coefficient of determination) on the given test data and labels.
        """
        return self._score(X, y, sample_weight)

    def report(self, width=900, height=1200):
        return self._report(width, height)

    def need_retrain(self, X, y, sample_weight=None, decrease=0.1):
        """Decides about model retraining based on new data.
        Arguments:
            X (numpy.ndarray or pandas.DataFrame):
                New data.
            y (numpy.ndarray or pandas.Series):
                True labels for X.
            sample_weight (numpy.ndarray or pandas.Series):
                Sample weights.
            decrease (float): The ratio of change in the performance used as a threshold for retraining decision.
                By default, it is set to `0.1` which means that if the performance of AutoML will decrease by 10%
                on new data then there is a need to retrain. This value should be set depending on your project needs.
                Sometimes, 10% is enough, but for some projects, it can be even lower than 1%.
            Returns:
                boolean: Decides if there is a need to retrain the AutoML.
        """
        return self._need_retrain(X, y, sample_weight, decrease)

In [6]:
from supervised.automl import AutoML

In [7]:
x_cols = [f for f in train.columns if "v" in f]

In [ ]:
automl = AutoML(
    total_time_limit=8*3600,
    optuna_time_budget=1800,
    mode="Optuna",
)
automl.fit(train[x_cols], train["target"])

AutoML directory: AutoML_3
Expected computing time:
Time for tuning with Optuna: len(algorithms) * optuna_time_budget = 10800 seconds
There is no time limit for ML model training after Optuna tuning (total_time_limit parameter is ignored).
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network']
AutoML will stack models
AutoML will ensemble availabe models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble', 'stack', 'ensemble_stacked']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 6 models
Optuna optimizes LightGBM with time budget 1800 seconds eval_metric logloss (minimize)


[I 2021-09-01 12:02:21,152] A new study created in memory with name: no-name-83e3ed4e-e90d-4fe4-81ef-7dc5da0cb346
[I 2021-09-01 12:03:35,096] Trial 0 finished with value: 0.47772858388546086 and parameters: {'learning_rate': 0.1, 'num_leaves': 1598, 'lambda_l1': 2.840098794801191e-06, 'lambda_l2': 3.0773599420974e-06, 'feature_fraction': 0.8613105322932351, 'bagging_fraction': 0.970697557159987, 'bagging_freq': 7, 'min_data_in_leaf': 36, 'extra_trees': False}. Best is trial 0 with value: 0.47772858388546086.
[I 2021-09-01 12:04:14,632] Trial 1 finished with value: 0.47766485406754905 and parameters: {'learning_rate': 0.0125, 'num_leaves': 30, 'lambda_l1': 0.09024841733204539, 'lambda_l2': 0.8785585624049705, 'feature_fraction': 0.5554201923798203, 'bagging_fraction': 0.7307773310574073, 'bagging_freq': 1, 'min_data_in_leaf': 37, 'extra_trees': True}. Best is trial 1 with value: 0.47766485406754905.
[I 2021-09-01 12:05:49,070] Trial 2 finished with value: 0.474681733542131 and parameter

1_Optuna_LightGBM logloss 0.462129 trained in 2707.12 seconds
Optuna optimizes Xgboost with time budget 1800 seconds eval_metric logloss (minimize)


[I 2021-09-01 13:17:58,861] A new study created in memory with name: no-name-3efcd74b-3e23-4447-bd81-2ea88acece82
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
[I 2021-09-01 13:18:31,328] Trial 0 finished with value: 0.4672982457504367 and parameters: {'eta': 0.1, 'max_depth': 10, 'lambda': 2.840098794801191e-06, 'alpha': 3.0773599420974e-06, 'colsample_bytree': 0.8613105322932351, 'subsample': 0.970697557159987, 'min_child_weight': 88}. Best is trial 0 with value: 0.4672982457504367.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
[I 2021-09-01 13:18:59,234] Trial 1 finished with value: 0.4688933890377344 and parameters: {'eta': 0.1, 'max_depth': 6, 'lambda': 0.0011239983523033718, 'alpha': 0.0003370920325799477, 'colsample_bytree': 0.30963791485116204, 'subsample': 0.8409786428569279, 'min_child_weight': 89}. Best is trial 0 with value: 0.4672982457504367.
ntree_limit is deprecated, use `iteration_range` or model slicing instead

ntree_limit is deprecated, use `iteration_range` or model slicing instead.
[I 2021-09-01 13:39:28,945] Trial 23 finished with value: 0.46457545513933646 and parameters: {'eta': 0.05, 'max_depth': 12, 'lambda': 9.284658490199415e-05, 'alpha': 0.783983177227589, 'colsample_bytree': 0.615085407164129, 'subsample': 0.9112166186884245, 'min_child_weight': 15}. Best is trial 22 with value: 0.46281519845414143.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
[I 2021-09-01 13:40:16,268] Trial 24 finished with value: 0.4660229741430812 and parameters: {'eta': 0.05, 'max_depth': 9, 'lambda': 0.0029709733627894257, 'alpha': 0.019270065820837144, 'colsample_bytree': 0.38297542180683386, 'subsample': 0.795033564550324, 'min_child_weight': 20}. Best is trial 22 with value: 0.46281519845414143.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
[I 2021-09-01 13:41:09,249] Trial 25 finished with value: 0.46412099834078957 and parameters: {'eta': 0.05,

...

[I 2021-09-01 14:05:00,735] A new study created in memory with name: no-name-f72f74a7-edcc-4717-ad3c-21a9e461101c
[I 2021-09-01 14:08:56,849] Trial 0 finished with value: 0.449237789725502 and parameters: {'learning_rate': 0.1, 'depth': 8, 'l2_leaf_reg': 7.7997800836072235, 'random_strength': 2.7259260601004898, 'rsm': 0.34881782962878705, 'min_data_in_leaf': 81}. Best is trial 0 with value: 0.449237789725502.
[I 2021-09-01 14:15:54,945] Trial 1 finished with value: 0.44997928717713326 and parameters: {'learning_rate': 0.05, 'depth': 6, 'l2_leaf_reg': 6.834661005427845, 'random_strength': 7.127020272701981, 'rsm': 0.43322567931135547, 'min_data_in_leaf': 57}. Best is trial 0 with value: 0.449237789725502.
[I 2021-09-01 14:17:58,985] Trial 2 finished with value: 0.4558110955655095 and parameters: {'learning_rate': 0.2, 'depth': 9, 'l2_leaf_reg': 3.648923350415333, 'random_strength': 6.1539617881809745, 'rsm': 0.16784311747867892, 'min_data_in_leaf': 37}. Best is trial 0 with value: 0.44

3_Optuna_CatBoost logloss 0.444697 trained in 11731.2 seconds
Optuna optimizes Neural Network with time budget 1800 seconds eval_metric logloss (minimize)


[I 2021-09-01 17:55:46,189] A new study created in memory with name: no-name-b3f85b0d-47e8-4964-aa9f-805d3b43c78f
[I 2021-09-01 17:57:22,076] Trial 0 finished with value: 0.4981166063094443 and parameters: {'dense_1_size': 22, 'dense_2_size': 63, 'learning_rate': 0.01, 'learning_rate_type': 'adaptive', 'alpha': 0.7645285581846926}. Best is trial 0 with value: 0.4981166063094443.
[I 2021-09-01 17:59:04,263] Trial 1 finished with value: 0.4961170628283237 and parameters: {'dense_1_size': 38, 'dense_2_size': 51, 'learning_rate': 0.01, 'learning_rate_type': 'adaptive', 'alpha': 0.8785585624049705}. Best is trial 1 with value: 0.4961170628283237.
[I 2021-09-01 18:07:22,155] Trial 2 finished with value: 0.4899473107370117 and parameters: {'dense_1_size': 39, 'dense_2_size': 62, 'learning_rate': 0.05, 'learning_rate_type': 'constant', 'alpha': 0.0012968444078239212}. Best is trial 2 with value: 0.4899473107370117.
[I 2021-09-01 18:10:14,009] Trial 3 finished with value: 0.4964992877288724 and

4_Optuna_NeuralNetwork logloss 0.480782 trained in 2441.46 seconds
Optuna optimizes Random Forest with time budget 1800 seconds eval_metric logloss (minimize)


[I 2021-09-01 19:05:02,234] A new study created in memory with name: no-name-005c02e4-f5e6-44ab-9578-c1ca10fdc1aa


In [ ]:
automl.report()